Okay, so since the contractor picked up the tab at Noah's, their initials should be in the database. The clues we have are:
- they got bagel and coffee
- their initials are JD
- this all happened in 2017
- the cleaner is across the street

I think the last is a red herring, since JD picked up the tab and we don't know where their address is. But worth keeping track of just in case!

### Initials JD

Let's start by getting the basic customer info we'll need: customerid, name, and phone.

In [1]:
import pandas as pd
customers = pd.read_csv('noahs-customers.csv',usecols=['customerid','name','phone'])
customers.head()

,customerid,name,phone
0,1001,Jack Quinn,805-287-8515
1,1002,David Powell,516-768-1652
2,1003,Carrie Green,727-209-0470
3,1004,Steven Miller,607-941-9563
4,1005,Christine Powers,212-759-9043


Split up the name to extract initials

In [2]:
names = customers['name'].str.split(' ',expand=True)
customers['firstname'] = names[0]
customers['lastname'] = names[2].mask(names[2].isnull(),names[1])
customers.head()

,customerid,name,phone,firstname,lastname
0,1001,Jack Quinn,805-287-8515,Jack,Quinn
1,1002,David Powell,516-768-1652,David,Powell
2,1003,Carrie Green,727-209-0470,Carrie,Green
3,1004,Steven Miller,607-941-9563,Steven,Miller
4,1005,Christine Powers,212-759-9043,Christine,Powers


In [3]:
customers['initials'] = customers['firstname'].str.slice(0,1) + customers['lastname'].str.slice(0,1)
customers.drop(['firstname','lastname'],axis=1,inplace=True)
customers.head()

,customerid,name,phone,initials
0,1001,Jack Quinn,805-287-8515,JQ
1,1002,David Powell,516-768-1652,DP
2,1003,Carrie Green,727-209-0470,CG
3,1004,Steven Miller,607-941-9563,SM
4,1005,Christine Powers,212-759-9043,CP


In [4]:
jds = customers[customers['initials']=='JD']
jds.head()

,customerid,name,phone,initials
443,1444,Jennifer Davis,315-356-9127,JD
513,1514,Joshua Downs,682-474-3747,JD
610,1611,Jacob Davis,516-521-7244,JD
741,1742,Jonathan Davis,585-510-8718,JD
753,1754,Jennifer Diaz,838-499-3728,JD


### The year was 2017

This all happened in 2017, so let's filter by customers with orders then. We'll have to join `jds` to the `noahs-orders.csv` dataset to use the `ordered` date. So let's import that dataset and merge.

In [5]:
orders = pd.read_csv('noahs-orders.csv',usecols=['orderid','customerid','ordered'])
orders.head()

,orderid,customerid,ordered
0,1001,4308,2017-01-31 00:32:19
1,1002,11683,2017-01-31 00:58:31
2,1003,5676,2017-01-31 01:34:40
3,1004,3097,2017-01-31 02:31:24
4,1005,10374,2017-01-31 02:46:09


In [6]:
jds_orders = jds.merge(orders,on='customerid')
jds_orders.head()

,customerid,name,phone,initials,orderid,ordered
0,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48
1,1611,Jacob Davis,516-521-7244,JD,4008,2017-03-03 10:23:46
2,1611,Jacob Davis,516-521-7244,JD,4674,2017-03-09 18:17:38
3,1611,Jacob Davis,516-521-7244,JD,5314,2017-03-16 03:29:20
4,1611,Jacob Davis,516-521-7244,JD,5852,2017-03-20 22:45:59


Now let's filter for orders from 2017

In [7]:
jds_orders['ordered'] = pd.to_datetime(jds_orders['ordered'],infer_datetime_format=True)
jds_orders_17 = jds_orders[jds_orders.ordered.dt.year == 2017]
jds_orders_17.head()

,customerid,name,phone,initials,orderid,ordered
0,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48
1,1611,Jacob Davis,516-521-7244,JD,4008,2017-03-03 10:23:46
2,1611,Jacob Davis,516-521-7244,JD,4674,2017-03-09 18:17:38
3,1611,Jacob Davis,516-521-7244,JD,5314,2017-03-16 03:29:20
4,1611,Jacob Davis,516-521-7244,JD,5852,2017-03-20 22:45:59


### The order included bagels

Let's find out which of our JDs ordered bagels and coffee. We'll need to pull in `orders_items` and `products`. The `orders_items` table is just an intermediary: we'll use `orderid` to connect to our existing `jds_orders_17` dataframe, and `sku` to connect to `products`.

In [8]:
orders_items = pd.read_csv('noahs-orders_items.csv',usecols=['orderid','sku'])
orders_items.head()

,orderid,sku
0,1001,COL0820
1,1002,TOY8907
2,1002,KIT5813
3,1002,KIT3981
4,1003,KIT7098


In [9]:
products = pd.read_csv('noahs-products.csv',usecols=['sku','desc'])
products.head()

,sku,desc
0,DLI0002,Smoked Whitefish Sandwich
1,PET0005,"Vegan Cat Food, Turkey & Chicken"
2,HOM0018,Power Radio (red)
3,KIT0034,Azure Ladle
4,PET0041,"Gluten-free Cat Food, Pumpkin & Pumpkin"


Now let's join `orders_items` to our JDs in 2017 dataset. This is just an intermediary table to get the product description in `products`.

In [10]:
with_items = jds_orders_17.merge(orders_items,on='orderid')
with_items.head()

,customerid,name,phone,initials,orderid,ordered,sku
0,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48,TOY4377
1,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48,COL4944
2,1611,Jacob Davis,516-521-7244,JD,4008,2017-03-03 10:23:46,KIT8811
3,1611,Jacob Davis,516-521-7244,JD,4674,2017-03-09 18:17:38,PET1894
4,1611,Jacob Davis,516-521-7244,JD,5314,2017-03-16 03:29:20,CMP4627


Now that we have `sku`, we can join to `products` to get descriptions of orders.

In [11]:
with_desc = with_items.merge(products,on='sku')
with_desc.head()

,customerid,name,phone,initials,orderid,ordered,sku,desc
0,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48,TOY4377,Mechanical Blocks
1,1611,Jacob Davis,516-521-7244,JD,1341,2017-02-03 11:34:48,COL4944,Noah's Jersey (white)
2,2297,James Davidson,516-515-6965,JD,30802,2017-11-26 19:00:40,COL4944,Noah's Jersey (white)
3,6122,Julie Melissa Duffy,347-716-8024,JD,25289,2017-10-02 13:57:06,COL4944,Noah's Jersey (white)
4,1611,Jacob Davis,516-521-7244,JD,4008,2017-03-03 10:23:46,KIT8811,Manual Onion Cup


Lastly, let's see which JD ordered a bagel

In [12]:
with_desc[with_desc.desc.str.contains('Bagel')]

,customerid,name,phone,initials,orderid,ordered,sku,desc
145,4164,Jeremy Davis,212-771-8924,JD,7409,2017-04-05 12:49:41,BKY5887,Sesame Bagel


Tada!